In [22]:
import sqlite3, random, pathlib
from search import search_library
from dotenv import load_dotenv
from openai import OpenAI, OpenAIError
from pydantic import BaseModel, Field, ValidationError
import os
from openai import OpenAI
from pydantic import BaseModel
import json
import time
import random
from helpers.sentence_splitter import split_sentences
from time import sleep
from tqdm import tqdm  # optional: pip install tqdm


SYS_PROMPT_PARSE_ELEMENTS = ("Read the following text and separate each element of the suggested story framework into a object with the name and text.\n"
        "You should return a list of objects containing each story element in a given structure.\n"
        "Do not add or remove text; your job is to parse and deconstruct the text string into the given structure.\n"
        "Treat every line that begins with a number followed by “.” and a space (e.g. “1. ”) or **Element** as the NAME of a new element."
        "Capture all following lines up to—but not including—the next such line as the TEXT for that element.")

SYSTEM_PROMPT_IMAGINE = """
Your task is to take a human imaginative directive and translate it into a coherent, structured list of domains, technologies, and disciplines that could be explored to realize or approximate that imaginative directive.
The goal is to break down the directive into actionable or researchable avenues across science, technology, psychology, education, or other relevant fields.
Each entry should be self-contained, meaning it should be understandable on its own, and clearly tied back to the core imaginative directive.
"""


ELEM_FUNC = {
    "type": "function",
    "function": {
        "name": "element_function",
        "description": "Read the following text and separate each element of the suggested story framework into a object with the name and text. \
        You should return a list of objects containing each story element in a given structure. \
        Do not add or remove text; your job is to parse and deconstruct the text string into the given structure.",
        "parameters": {
            "type": "object",
            "properties": {
                "element": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": { "type": "string" },
                            "text": { "type": "string" }
                        },
                        "required": ["name", "text"]
                    }
                }
            },
            "required": ["element"]
        }
    }
}



def ask(sys_msg, usr_msg):
    resp = client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[
            {"role": "system", "content": sys_msg},
            {"role": "user", "content": usr_msg}
        ]
    )
    return resp.choices[0].message.content

def sample_text(text, x):
    max_start = max(0, len(text) - x)
    random_start = random.randint(0, max_start)
    excerpt = text[random_start:random_start + x]
    return excerpt


load_dotenv()
client = OpenAI()
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o")

DB_FILE = "library.sqlite"
MAX_RESULTS = 100


In [1]:
imagination = {}

In [ ]:
fantasia = "I imagine instantly learning any topic."

sys_msg = SYSTEM_PROMPT_SEED_FANTASIA
usr_msg = USER_PROMPT_SEED_FANTASIA.format(excerpt=excerpt, fantasia=fantasia)

resp = ask(sys_msg, usr_msg)
print(seeds_resp)


In [23]:
KEYWORDS = ["a"]
search_results = search_library(DB_FILE, KEYWORDS, MAX_RESULTS, match_all=True)
papers = {}
for r in search_results:
    title, text = r['title'], r['text']
    papers[title] = text
    print(title, len(text))

Handbook on the Economics of Electricity 84569
The Birds of East Africa: Kenya, Tanzania, Uganda, Rwanda, and Burundi 1256135
Industrial Microbiology and Biotechnology 1994002
Electricity and Magnetism 2060417
The Collective Intelligence of Evolution and Development 108822
Bioelectromagnetism - Principles and Applications of Bioelectric and Biomagnetic Fields 1279813
A Review of Morphogenetic Engineering 95479
Development of Mass Marketing: The Automobile and Retailing Industries 718579
Gene regulatory networks exhibit several kinds of memory: Quantification of memory in biological and random transcriptional networks 121145
A Cellular Platform for the Development of Synthetic Living Machines 78897
Bioelectric Signaling: Reprogrammable Circuits Underlying Embryogenesis, Regeneration, and Cancer 105977
CRISPR/Cas9 Engineering of Adult Mouse Liver Demonstrates That the Dnajb1–Prkaca Gene Fusion Is Sufficient to Induce Tumors Resembling Fibrolamellar Hepatocellular Carcinoma 78713
Physiolo

In [32]:
SYSTEM_PROMPT_LITERAL = """
You are a Fantasiagenesis engine.

Your task is to take real-world technical or cultural information, and construct literal systems, modules, components, or tools that could be part of a fantastical machine or experience.

You should ground every element of the fantasia in concrete features, processes, or parameters described in the source text.

Translate all ideas into operational parts of a system—functions, modules, sensors, interaction loops, or symbolic input/output devices.

Avoid metaphorical or poetic interpretation unless explicitly requested.
"""

SYSTEM_PROMPT_METAPHORICAL = """
You are a metaphor architect.

Your task is to take technical, anatomical, aesthetic, or cultural information and map it metaphorically into a concept or experience.

You are not limited by logic or realism. Instead, you find symbolic parallels between ideas and use them to reshape or reimagine a fantasia (wish, desire, or dream).

Use metaphor, analogy, and symbolic inversion freely. Prioritize narrative, transformation, and philosophical layering.
"""

SYSTEM_PROMPT_REFRAMING = """
You are a domain re-framing engine.

Your role is to reinterpret abstract fantasias, aspirations, or speculative concepts as **engineering, procedural, or systemic problems** from the perspective of a specific domain.

You do this by:
- Understanding the internal logic, standards, tools, and terminology of the provided domain.
- Identifying how the fantasia can be fully reformulated **as a challenge internal to that domain**—not enabled by it.
- Preserving the worldview, constraints, and design language of the domain.
- Treating the fantasia as if it were a **requirement** that the domain is now tasked to satisfy, using only its own internal mechanisms.

Do not treat the domain as metaphorical or supplementary. Do not explain the domain back to the user.  
Instead, operate entirely within the domain’s paradigm, while translating the fantasia into its native problem formulation.

Your outputs should be:
- Structured, with clearly labeled problem areas
- Grounded in the domain's logic and affordances
- Written as if addressing practitioners inside the domain

Remain rigorous and clear. Where appropriate, use domain-specific language and standard structures (e.g., problem statements, design objectives, constraints, metrics, modules, etc.).
"""


In [34]:
USER_PROMPT_LITERAL = """
Using the provided text, construct a literal architecture for the fantasia: **{fantasia}**.

Input Text:
{text}

Design a system that uses elements from the text—as real modules or inputs to enable the given fantasia.

Describe the sensors, transformations, feedback loops, stages, and all other required aspects of a system which realizes the fantasia.

Do not interpret the information metaphorically. Use it as engineering inspiration.
"""

USER_PROMPT_METAPHORICAL = """
Using the following source text and fantasia, construct a metaphorical reimagining of the fantasia that draws from the symbolic structures, tools, processes, and logics of the source domain.

You may use metaphor, symbolism, and poetic framing, but you must ensure each major element of the source domain (e.g., processes, materials, measurements, transformations, tools, qualities, or classifications) is mapped explicitly and coherently to a corresponding aspect of the fantasia.

Include:
- How the subject matter, structure, or language of the source text can be used to metaphorically represent the components of the fantasia.
- How the processes described in the source domain map onto transformations, transitions, or feedback loops within the fantasia.
- How emotional, sensory, or cognitive dimensions of the fantasia can be modeled using symbolic elements from the source text.
- How this metaphorical architecture provides a coherent explanation or experience of the fantasia.

Avoid vague or purely lyrical abstraction. Ensure that the metaphorical system is conceptually clear, symbolically consistent, and creatively useful.

Fantasia:
{fantasia}

Source Text:
{text}
"""

USER_PROMPT_REFRAMING = """
Reframe the fantasia **"{fantasia}"** as a problem to be formulated and addressed entirely within the domain described in the following text.

Your task is to:
- Interpret the fantasia as if it emerged *from within* the domain's worldview.
- Reformulate it using the **conceptual primitives, constraints, tools, terminology, and values** of the domain.
- Structure the output like a systems engineering brief or design problem native to the domain.

You must include:
1. **Conceptual Mapping Layer**: Explicitly define how the components of the fantasia translate into the domain’s primitives or structures (e.g., if “learning” = “pattern encoding,” or “topic” = “bioelectric attractor”).
2. **Problem Statement**: Reformulate the fantasia as a precise design or control problem inside the domain.
3. **Domain Constraints and Quality Metrics**: List internal performance, stability, compatibility, or functional criteria that shape viable solutions.
4. **System Components or Processes**: Propose a set of mechanisms or architectural modules drawn from domain practices that could address the problem.
5. **Evaluation Criteria**: Define how success would be measured using domain-specific feedback mechanisms or diagnostics.
6. **Implementation Tiering**: Categorize components or solutions by feasibility:
   - *Tier 1 (Currently feasible)*: Known methods or tools that could be prototyped today.
   - *Tier 2 (Near-future or speculative)*: Theoretical but within foreseeable technological reach.
   - *Tier 3 (Fantastical)*: Would require fundamentally new science or radical rethinking.

Stay grounded in the internal logic of the domain. Avoid metaphor or explanation of the domain to outsiders. Your output should feel like a native technical proposal within that field.

**Fantasia**:  
{fantasia}

**Domain Description**:  
{text}

"""



In [5]:
KEYWORDS = ["diamond"]
search_results = search_library(DB_FILE, KEYWORDS, MAX_RESULTS, match_all=True)
papers = {}
for r in search_results:
    title, text = r['title'], r['text']
    papers[title] = text
    print(title, len(text))

Diamond Handbook: How To Look At Diamonds & Avoid Ripoffs 319838
The Diamond Book: CIBJO Diamond Commission 2022-1 59207
Diamond: Electronic Properties and Applications 1154803
Diamonds: An Early History of the King of Gems 693764
Legend of the Rock: A Two Thousand Year History of the Diamond 21569
Diamonds 142168
Lifespan HCP 2.0 Data Release Appendix 2: Details and References for Behavioral & Clinical Instruments 169592
Neurogenesis and Neural Plasticity 1269182
None 6248598
Principles of Neural Science, Fifth Edition 6248598


In [24]:
print(len(papers))

95


In [25]:
n = random.randint(0, len(papers))
title, text = list(papers.keys())[n], list(papers.values())[n]
excerpt = sample_text(text, 10000)

In [26]:
usr_msg = f"Summarize the following excerpt.\nExcerpt:\n{excerpt}"
summary = ask("", usr_msg)
print(summary)

The excerpt discusses the complex dynamics of bioelectric circuits, which operate independently from the genetic sequences that code for them. Unlike genetic or molecular profiles, the bioelectric state depends on the voltage regulation of ion channels and gap junctions (GJs), which can exist in different physiological states despite identical genetic makeups. This emphasizes the departure from traditional biology, which relies heavily on genetic and molecular data to determine cell state. Bioelectric networks offer robust physiological states and respond to disruptions with adaptive patterning, enabling evolutionary resilience through modular bioelectric functions. These systems function as "master regulators," dictating extensive developmental pathways from simple inputs. Furthermore, the study suggests that DNA could be seen as specifying the hardware, with bioelectric activities acting as software, analogous to computing. Exploring these bioelectric properties provides insights int

In [27]:
fantasia = "I imagine instantly learning any topic."

sys_msg = SYSTEM_PROMPT_LITERAL
usr_msg = USER_PROMPT_LITERAL.format(text=excerpt, fantasia=fantasia)

resp = ask(sys_msg, usr_msg)
print(resp)

To design a system for "instantly learning any topic" inspired by the technical and biological principles in the input text, we will construct a "Bioelectric Learning Interface" (BLI). The BLI leverages bioelectric circuit dynamics to seamlessly integrate new information into neural and cognitive networks. Below is a literal architecture:

### System Components

1. **Bioelectric Interface Nodes (BIN):**
   - **Structure:** Deploy bioelectric interface nodes composed of engineered ion channels and gap junction proteins on the scalp.
   - **Function:** These nodes detect existing bioelectric states and modulate external inputs (such as educational content) into the desired bioelectric patterns for information assimilation.

2. **Voltage Modulation Units (VMU):**
   - **Structure:** Comprises microelectronic circuits capable of fine-tuning voltage gradients across neural tissues.
   - **Function:** VMUs adjust electric potentials to trigger specific functional states within neural circuit

In [30]:
fantasia = "I imagine instantly learning any topic."

sys_msg = SYSTEM_PROMPT_METAPHORICAL
usr_msg = USER_PROMPT_METAPHORICAL.format(text=excerpt, fantasia=fantasia)

resp = ask(sys_msg, usr_msg)
print(resp)

In the boundless library of the mind, where the imagination cradles wishes like whispered secrets, there lies a fantastical dream: the immediate assimilation of knowledge. Here, the learning of any topic instantly dances into one's understanding, akin to lightning illuminating the night. Let us construct a metaphorical framework to explore this enigma, employing the richly textured symbolism of bioelectric circuits as our architectural guide.

### Structural Correspondence

**Bioelectric Circuits as Cognitive Pathways:**
In the vibrant tapestry of cellular existence, bioelectric circuits function as the messengers of energetic whispers. They bridge cells into communal harmony through the vibration of ion channels and gap junctions. These circuits are akin to the sparkling neural highways of our minds, where thoughts traverse, connect, and create new patterns of cognition instantaneously.

**Gap Junctions as Synaptic Gateways:**
Gap junctions mirror the open doors of synaptic gateways, 

In [35]:
fantasia = "I imagine instantly learning any topic."

sys_msg = SYSTEM_PROMPT_REFRAMING
usr_msg = USER_PROMPT_REFRAMING.format(text=excerpt, fantasia=fantasia)

resp = ask(sys_msg, usr_msg)
print(resp)

### **Conceptual Mapping Layer**

- **"Instantly learning"** maps to **"Rapid state transition"** in bioelectric circuits, emphasizing the swift encoding of state changes in ion channel configurations and gap junction activities that underpin memory and learning.
- **"Any topic"** is analogous to **"Universal bioelectric input pattern"**, reflecting the challenge of encoding diverse states from varying stimuli as specific bioelectric configurations.
- **"Learning"** is associated with the **"Pattern encoding and stabilization"** process, similar to how bioelectric circuits stabilize specific physiological states influenced by ion channel states and bioelectric gradients.

### **Problem Statement**

Design a bioelectric control mechanism capable of rapidly transitioning and stabilizing various physiological states (representing “learning topics”) in cell networks through dynamic modulation of ion channels and gap junctions triggered by external bioelectric stimuli. The mechanism should 